## Paquetes

In [1]:
import pandas as pd
import numpy as np
import re
import time

In [61]:
import requests as req
from bs4 import BeautifulSoup as bs

In [3]:
import sys
from src.scraping_functions import *

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
opciones.add_experimental_option("detach", True)
path = "/mnt/d/Google Drive/Iron hack/Instalaciones/chromedriver.exe"

## Importar  y limpiar csv

In [79]:
data = pd.read_csv("data/residential-stay-in-covid19.csv",encoding = "ISO-8859-1")

In [80]:
data.shape

(91933, 3)

In [81]:
data.sample(5)

,Entity,Day,Increase in Residential Stay
79514,Switzerland,25-04-2021,6.429
29098,Gabon,26-10-2021,-13.286
32209,Guatemala,07-09-2020,16.000
84917,Uganda,20-06-2020,15.857
19778,Costa Rica,18-09-2021,4.714


In [82]:
data.isna().sum()

Entity                          0
Day                             0
Increase in Residential Stay    0
dtype: int64

In [83]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91933 entries, 0 to 91932
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Entity                        91933 non-null  object 
 1   Day                           91933 non-null  object 
 2   Increase in Residential Stay  91933 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.1+ MB


In [84]:
nuevas_columnas = {"Entity": "country", "Day":"date"}
data.rename(columns=nuevas_columnas, inplace=True)

In [85]:
data["date"]=pd.to_datetime(data["date"],dayfirst=True)

In [86]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91933 entries, 0 to 91932
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   country                       91933 non-null  object        
 1   date                          91933 non-null  datetime64[ns]
 2   Increase in Residential Stay  91933 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 2.1+ MB


In [87]:
eeuu=data[data.country=="United States"]

In [88]:
eeuu.date.min()

Timestamp('2020-02-17 00:00:00')

In [89]:
eeuu.drop(columns = ["country"],axis=1,inplace= True)

/tmp/ipykernel_2114/2038731044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeuu.drop(columns = ["country"],axis=1,inplace= True)


In [90]:
eeuu.head(5)

,date,Increase in Residential Stay
86935,2020-02-17,1.000
86936,2020-02-18,1.000
86937,2020-02-19,0.800
86938,2020-02-20,0.833
86939,2020-02-21,0.714


## Scrapeo

In [94]:
dict_tablas={}
dict_df={}
for e in ["NFLX","DIS","AMC"]:
    dict_tablas[e] = yahoo_finance(e, 771, 3, 1, "history")    
    dict_df[e]=pd.DataFrame([i.split(" ") for i in dict_tablas[e].split("\n")]
                            ,columns=["month","day","year", f"open_{e}", f"high_{e}", f"low_{e}", f"close_{e}", f"adj_close_{e}", f"volume_{e}"])

In [95]:
for e in list(dict_df.keys()):
    dict_df[e]['date'] = dict_df[e]["year"]+" "+dict_df[e]["month"]+" "+dict_df[e]["day"]
    dict_df[e]['date'] = dict_df[e]['date'].apply(lambda x : x.rstrip(","))
    dict_df[e]['date'] = pd.to_datetime(dict_df[e]['date'], format='%Y %b %d')
    dict_df[e].drop(columns = ["year","month","day",f"open_{e}", f"high_{e}", f"low_{e}", f"adj_close_{e}", f"volume_{e}" ]
                    ,axis=1,inplace= True)
    dict_df[e] = dict_df[e].iloc[:,[1,0]]


In [96]:
dict_df["NFLX"].sample(5)

,date,close_NFLX
145,2021-07-12,537.31
492,2020-02-25,360.09
461,2020-04-08,371.12
261,2021-01-25,556.78
120,2021-08-16,517.92


In [97]:
tabla = eeuu.merge(dict_df["NFLX"], left_on='date', right_on='date').merge(dict_df["DIS"], left_on='date', right_on='date').merge(dict_df["AMC"], left_on='date', right_on='date')

In [100]:
tabla.to_csv("data/eeuu.csv",index=False)

In [101]:
tabla.sample(5)

,date,Increase in Residential Stay,close_NFLX,close_DIS,close_AMC
143,2020-09-09,7.571,500.19,133.36,6.26
57,2020-05-07,14.714,436.53,105.57,3.99
34,2020-04-03,18.429,361.76,93.88,2.27
286,2021-04-06,6.286,544.53,189.73,10.20
17,2020-03-11,-0.429,349.92,105.51,3.64
